## Imports

In [ ]:
import sys
import os 
import warnings
import numpy as np

pkg_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(pkg_path)

np.set_printoptions(threshold=np.inf, suppress=True, linewidth=np.inf)
warnings.filterwarnings(action='ignore', category=UserWarning)

import neuroseq.addsrc
# from neuroseq.common.config import RuntimeConfig, Backends
from neuroseq.common.config import *
from neuroseq.core.logging import log

In [ ]:
RuntimeConfig.backend = Backends.NEST

In [ ]:
if RuntimeConfig.backend == Backends.BRAIN_SCALES_2:
    from neuroseq.brainscales2.network import SHTMTotal
    from neuroseq.brainscales2.hardware import hardware_initialization
    hardware_initialization()
elif RuntimeConfig.backend == Backends.NEST:
    from neuroseq.nest.network import SHTMTotal

from neuroseq.core.parameters import NetworkParameters
from neuroseq.core.helpers import Latex
from neuroseq.core.data import load_yaml

## Config

In [ ]:
experiment_type = ExperimentType.EVAL_MULTI
experiment_id = "bss2_s1"
experiment_num = 2

titles = {
    "experiment": "Experiment",
    
}

## Load Parameters

In [ ]:
p = NetworkParameters(network_type=SHTMTotal)
# p.load_experiment_params(experiment_type=ExperimentType.EVAL_MULTI, 
                         # experiment_id=experiment_id, 
                         # experiment_num=experiment_num)

# load default params for yaml generation
p.load_default_params()
p_dict = p.dict()

p_conv_path = join(RuntimeConfig.Paths.package, "config")
p_conv = load_yaml(p_conv_path, "shtm2bss_config_parameter-conversion.yaml")


## Print all parameters in YAML syntax

In [ ]:
NetworkParameters.print_parameter(p_dict)

## Print all parameters from YAML to Latex table

In [ ]:
cur_title_org = ""
for param_name, param_values in p_conv["parameters"].items():
    new_title = param_name.rsplit('.', maxsplit=1)[0]
    if cur_title_org != new_title:
        if '.' in new_title:
            cur_title = ' '.join(new_title.split('.')[::-1])
            cur_title = cur_title.capitalize()
        else:
            cur_title = new_title.capitalize()
        Latex.print_title(cur_title)
        cur_title_org = new_title
    
    value = p.get_custom_param(param_name)
    if param_name == "experiment.sequences":
        if len(value[0]) > 3:
            value = "$S_2$"
        else:
            value = "$S_1$"
    else:
        value = str(value).replace('_', '\\_')
    Latex.print_row([param_values["name"], str(value), param_values["description"]])